In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/RSO Activities/DAIS/DAIS FALL 2021/Project Term 2/조성근, 이진호/Codes") 
#os.chdir("/content/drive/MyDrive/DAIS FALL 2021/Project Term 2/조성근, 이진호/Codes")

# Load the Data required for GNN

In [ ]:
from gnn_preprocess import *
A, input_feats, input_labels, num_classes, train_mask, test_mask = load_gnn_data()

[[116.41402   62.37332   48.501354 ...  64.320076  62.056103  65.41077 ]
 [ 62.37332  139.60555  124.27874  ...  28.512691  28.590662  34.40452 ]
 [ 48.501354 124.27874  131.95673  ...  31.995268  29.975834  35.186543]
 ...
 [ 64.320076  28.512691  31.995268 ...  97.92377   94.67897  101.13614 ]
 [ 62.056103  28.590662  29.975834 ...  94.67897   94.18068   99.59302 ]
 [ 65.41077   34.40452   35.186543 ... 101.13614   99.59302  108.96373 ]]
[[0.77873594 0.42084992 0.32898244 ... 0.43374237 0.41874915 0.4409655 ]
 [0.42084992 0.9323225  0.8308203  ... 0.19660692 0.19712329 0.23562571]
 [0.32898244 0.8308203  0.881668   ... 0.21967039 0.20629662 0.24080469]
 ...
 [0.43374237 0.19660692 0.21967039 ... 0.65628374 0.634795   0.67755777]
 [0.41874915 0.19712329 0.20629662 ... 0.634795   0.63149506 0.66733843]
 [0.4409655  0.23562571 0.24080469 ... 0.67755777 0.66733843 0.7293962 ]]
[[1 1 0 ... 1 1 1]
 [1 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [1 0 0 ... 1 1 1]
 [1 0 0 ... 1 1 1]
 [1 0 0 ... 

In [ ]:
print(A)
print('\ninput_feats:', input_feats.shape)
print('input_labels:', input_labels)
print('num_classes:', num_classes)
print('train_mask:', train_mask)
print('test_mask:', test_mask)

[[1 1 1 ... 0 1 1]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]
 ...
 [0 0 0 ... 1 0 0]
 [1 0 1 ... 0 1 1]
 [1 0 0 ... 0 1 1]]

input_feats: (58, 128)
input_labels: [2 5 5 5 5 5 5 5 5 5 5 5 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 2]
num_classes: 6
train_mask: [1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1]
test_mask: [0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0]


## **Node Classification**

### **Setup**

In [ ]:
# Install the required dgl library.
pip install dgl

     |████████████████████████████████| 4.4 MB 4.4 MB/s 


In [ ]:
# Import necessary libraries.
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


### **Implementation**

In [ ]:
# Convert the derived adjacency matrix into coordinate format (COO) sparse matrix.
coo = sp.coo_matrix(A)

# Create a graph object using sparse adjacency matrix.
src_ids = torch.tensor(coo.row)
dst_ids = torch.tensor(coo.col)
g = dgl.graph((src_ids, dst_ids))

# Set up node features with the derived elements above.
g.ndata['feat'] = torch.Tensor(input_feats)  # Vector representation of each picture (128 dimension).
g.ndata['label'] = torch.tensor(input_labels, dtype=torch.long)  # Assigned label for each person.
g.ndata['train_mask'] = torch.tensor(train_mask, dtype=torch.bool)
g.ndata['test_mask'] = torch.tensor(test_mask, dtype=torch.bool)

In [ ]:
# A usual GCN model for node classification.
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, num_classes)

In [ ]:
# Build a train model for node classification.
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        m = torch.nn.Softmax()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, train acc: {:.3f}, test acc: {:.3f}'.format(
                e, loss, train_acc, test_acc))

# Implement the train model above.
model = GCN(g.ndata['feat'].shape[1], 16, num_classes)
train(g, model)

In epoch 0, loss: 2.315, test acc: 0.188
In epoch 5, loss: 1.411, test acc: 0.250
In epoch 10, loss: 1.174, test acc: 0.375
In epoch 15, loss: 1.010, test acc: 0.562
In epoch 20, loss: 0.873, test acc: 0.625
In epoch 25, loss: 0.729, test acc: 0.750
In epoch 30, loss: 0.589, test acc: 0.750
In epoch 35, loss: 0.462, test acc: 0.875
In epoch 40, loss: 0.351, test acc: 1.000
In epoch 45, loss: 0.258, test acc: 1.000
In epoch 50, loss: 0.191, test acc: 0.938
In epoch 55, loss: 0.146, test acc: 0.938
In epoch 60, loss: 0.115, test acc: 0.938
In epoch 65, loss: 0.093, test acc: 0.938
In epoch 70, loss: 0.078, test acc: 0.938
In epoch 75, loss: 0.066, test acc: 0.938
In epoch 80, loss: 0.057, test acc: 0.938
In epoch 85, loss: 0.050, test acc: 0.938
In epoch 90, loss: 0.044, test acc: 0.938
In epoch 95, loss: 0.039, test acc: 0.938
